# People Analytics
## Predicting Employee Attrition with R

<img src="images/office-and-workers-in-barcelona-spain.jpg" />

<br>

*In this notebook, we will walk through solving a classification problem. Classification is a type of Machine Learning where we identify which set of categories a person or observation might belong to. We use a set of "training" data containing observations whose category is known and then build a model to make judgements about what classification we might put other people (or instances) into. Classification is an example of pattern recognition, we look for patterns in the data to help us predict what future patterns might be.* 

### Table of Contents
<a href="#sectionhelp"> Getting Help </a>

<a href="#sectioncase">Case Study: Employee Attrition at IBM</a>

<a href="#sectiondata"> The Dataset </a>

<a href="#sectionlogistic"> Logistic Regression </a>

<ul>
    <li> <a href="#sectionex1">Example 1: Predicting Based on Monthly Income </a> </li>
    <li> <a href="#sectionex2">Example 2: Predicting Based on Monthly Income, Work-Life Balance, and Years at Company</a> </li>
    <li> <a href="#sectionmake">Make Your Own Model</a></li>
    </ul>

<a href="#sectionsubmit"> Submitting your assignment </a>


### Getting Help <a id='sectionhelp'></a>

As you go through this assignment, you might have questions or technical errors come up. A few troubleshooting tips can help you solve most Jupyter problems:

1. **re-run your code cells**: if you get an "object '___' not found" error under a code cell, you may have forgotten to run a cell above it. Try re-running previous cells in order, from top to bottom, or go to the Cell menu and click "Run all" or "Run all above".
2. **try a different browser**: either Firefox or Chrome should work with minimal issues.
3. **restart your notebook kernel**: the "kernel" is the program that runs Jupyter. Restart it by going to the Kernel menu and clicking "Restart". Note: this will make Jupyter "forget" any commands you previously gave it; you will need to re-run previously run code cells.
4. **restart your server**: if all else fails, try clicking "Control Panel" at the top right, then click "Stop my server" followed by "Start my server". Like restarting your kernel, this will require you to re-run any code cells you had already executed.

There are also several resources available to help you at any time:

* **Your classmates**: it's not uncommon for students to enter this course with some familiarity with programming, R, and/or Jupyter.
* **Your course reader**: reach out to Daniela Perez via email or bCourses with questions on programming and Jupyter.
* **The Data Peer Consultants**: this group of Berkeley students provides free online drop-in consultations for any Berkeley students working with Jupyter or data analysis. See their schedule and attend a consultation at [https://data.berkeley.edu/ds-peer-consulting](https://data.berkeley.edu/ds-peer-consulting).
* **The internet**: the first thing most professional data analysts do when they run into technical issues is to Google them. Jupyter and R are both widely-used and well-documented- the solution to your problem may already be online!


## <a id= "sectioncase">Case Study: Employee Attrition at IBM </a>

*Employee attrition* refers to a decrease in employees at a company, caused by resignations, retirements, or the elimination of job positions. It is important for businesses to anticipate attrition so they can keep costs low and properly distribute workloads.

Today, we'll try to predict whether or not an employee will leave their job using [a data set created by IBM](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/home). Because employee data is confidential, IBM's data set is fictional- it does not represent real employees. However, the data and the problem are structured very similarly to how actual companies such as IBM are approaching this very real problem. In analytics, the focus of the problem, study, or experiment is often called the <b>response</b>, <b>explanatory</b>, or the <b>dependent</b> variable. In this example, attrition is our response or dependent variable. Any variables that we may use to predict attrition would be our <b>independent</b> variable(s).

<div class="alert alert-info">
    <b>NOTE</b>: we've made some changes to the data set to make this assignment simpler- things like renaming values, dropping confusing features, and adding definitions for columns. Please refer to the data dictionary below and <b>not</b> the one at the link when completing this assignment.
    </div>
    
Run the following cell to load the R-packages we will use to analyze the data. These packages provide a starting point that will allow us to process data without having to write code from scratch.       


In [ ]:
# Run this cell to install the necessary software
install.packages("fmsb")
library(corrplot)
library(ggplot2)
library(plyr)
library(fmsb)


## <a name="sectiondata"> The Dataset </a>


Run the following cell to load the data.

In [ ]:
# load the dataset
hr = read.csv("data/HR-Employee-Attrition.csv")

# show the first 6 rows of the dataset
head(hr)

This data set contains a wealth of information that could be used to predict attrition. In fact, there is so much information that the rows and columns have been truncated in the notebook.

We can see the dimensions of the dataset (number of rows and columns, respectively) with the `dim` function.

In [ ]:
# show the size of the dataset
dim(hr)

The first number is the number of rows and it shows how many employees are available on the dataset. The second number is the number of columns, which indicates how many variables we have data on for these employees. We can view all of the column names by using the `names` function.

In [ ]:
# view all column names
names(hr)

The source website included the **data dictionary**: the information about each feature (column) contained in the data set. We've reproduced the data dictionary here.


* `Age`: employee's age
* `Attrition`: the **response variable**. 'Yes' if employee left IBM due to normal life circumstances, 'No' if the employee stayed with IBM
* `BusinessTravel`: the amount of travel the employee does for business. 0 (no travel), 1 (rare travel), or 2 (frequent travel)
* `Department`: employee's home department. 'Sales', 'Research & Development', or 'Human Resources'
* `DistanceFromHome`: number of miles the employee has to travel to work
* `Education`: employee's highest level of education. 1 (below college), 2 (some college), 3 (bachelor's degree), 4 (master's degree), 5 (doctorate)
* `Gender`: 1 (female) or 0 (male)
* `JobRole`: 'Sales Executive', 'Research Scientist', 'Laboratory Technician', 'Manufacturing Director', 'Healthcare Representative', 'Manager', 'Sales Representative', 'Research Director', or 'Human Resources'
* `JobSatisfaction`: the employee's ranking of their satisfaction with their job. Minimum is 1, maximum is 4.
* `MaritalStatus`: 'Single', 'Married', or 'Divorced'
* `MonthlyIncome`: in dollars
* `NumCompaniesWorked`: the total number of companies the employee has worked at in their lifetime
* `OverTime`: whether or not the employee is eligible for overtime. 1 (yes) or 0 (no)
* `PercentSalaryHike`: for the employee's most recent salary increase, the percentage by which their salary increased
* `WorkLifeBalance`: employee's rating of their work-life balance. 1 (bad), 2 (good), 3 (better), 4 (best)
* `YearsAtCompany`: number of years employee has worked for IBM
* `YearsInCurrentRole`: number of years employee has worked for IBM in their current position

You saw in the data that our response variable for attrition used the words 'Yes' or 'No' to indicate whether the employee left the organization. In order to conduct our classification exercise we will need this data to be in the form of numbers. In fact, in the type of analysis we will be using today (logistic regression), we can only use variables that are numeric in value. So, we will need to 'transform' the attrition data into numbers and will only select the other variables that contain numbers. 

In the following cell we will add a new column (`AttritionInt`) to the dataset: the Attrition variable stored as an integer (1 if the employee left IBM, 0 if the employee stayed). This will be the actual response variable our models will use to predict attrition.

In [ ]:
# add a data frame column with Attrition coded as integers: 1 for "Yes" and 0 for "No"
# this will allow us to make plots
hr$AttritionInt = as.numeric(as.factor(hr$Attrition)) - 1

# select only columns with numeric data
numeric_data = hr[,sapply(hr,is.numeric)]
head(numeric_data)

### Exploratory Analysis

**Exploratory Data Analysis (EDA)** can help us get a sense of what data each column contains.

The `summary` function displays summary statistics for each column. For **quantitative variables**, `summary` will show the minimum, maximum, mean, and quartiles. For **categorical variables**, `summary` shows the possible values and frequencies for each.

In [ ]:
# show summary statistics for the data
summary(hr)

**Data visualization** can also be helpful to find relationships. **Box plots** show the distribution of values within a particular column of numerical data through their quartiles (thin horizontal lines on the next graph). The spacings between the different parts of the box indicate the degree of dispersion in the data. Outliers may be plotted as individual points.

In [ ]:
# replace the ... with the name of the column you want to visualize
# capitalization counts!
my_column = "..."

# run this cell to see the distribution of values for your column
boxplot(hr[,my_column],main=my_column)

**Correlation plots** show the strength of linear relationships between variables. Each cell on the plot shows the correlation between 2 variables: positive correlations are shown in blue and negative correlations are shown in red. The intensity of the color shows the strength of the correlation. Each element on the diagonal is the correlation of a variable with itself, which is always equal to 1. In this case we will use circles to highlight variables that have no correlation. 

In [ ]:
# create a correlation plot for quantitative variables
correlations = cor(numeric_data)
corrplot(correlations, method="circle",type='upper',na.label = "o")

<div class="alert alert-warning">
    <b>QUESTION</b>: Which variables do you think will be most useful to predict attrition, and what do you base that on (EDA, prior knowledge, etc)? Do you see any outliers? Based on the data dictionary and your own knowledge, what are some possible sources of bias in the dataset that might affect this analysis?\
    </div>


*YOUR ANSWER HERE* (double click)

## <a class="sectionlogistic"> Logistic Regression </a>

**Logistic Regression** is used to model the probability associated to binary events, such as win/lose, pass/fail or attrition/no attrition. Logistic regressions take numeric variables as input and build a statistical model by expressing the dependent variable as a linear combination of the independent variables using the exponential function. In the following example, we will use a logistic model to calculate the probability of attrition for an employee.

### <a class="sectionex1"> Example 1: predict by monthly income rate (univariate logistic regression) </a>

Let's try predicting attrition based on the employee's monthly income.

The first task is to write out the formula and save it as a variable. In this example, the name of the formula variable is `univar_formula`. The formula itself is to the right of the `=` and has the syntax:

$$ \text{response variable} \sim \text{predictor variable} $$

That is, the name of the variable we want to predict is on the left of the tilde, and the name of the variable we're using to predict attrition is on the right.

In [ ]:
# write the formula for univariate logistic regression
univar_formula = AttritionInt ~ MonthlyIncome

R provides a function called `glm` (for Generalized Linear Model) that will fit the data. The function takes three arguments:
* the formula (symbolic description of the model)
* `data` = the name of the data frame that holds our data
* `family` = a description of the error distribution and link function to be used in the model. For logistic regression, we want `binomial`

Run the following cell to fit the model.

In [ ]:
# fit the model
logreg_single = glm(univar_formula, 
              data = hr, # the dataset
              family = binomial) # the type of model to use

The `summary` function displays some information about the fit model. We will focus on the coefficients associated with the intercept and the independent variable. Similarly to the linear regression case, the intercept allows us to calculate the expected value of the probability when the independent variables are equal to zero and the coefficient of the independent variable shows us how much the outcome probability changes when the independent variable changes. The `summary` provides the following information: 
* **Estimate**: value of the coefficient obtained after training the model.
* **Standard Error**: summarizes the deviation between the predicted values and the real observations. It can be used to build confidence intervals for the predictions.
* **Z value**: normalized value used to run a hypothesis and check if there is a statistically significant relationship between the independent and dependent variable. The significance (next bullet) will allow us to check this relationship more directly. 
* **Significance (Pr(>|z|))**: obtained by checking the Z value on a standard normal table. If this value is below 5%, then we can affirm we are 95% confident that the coefficient of the independent variable is different from zero (i.e. there is a relationship between dependent and independent variables).   




In [ ]:
# show some summary information about the fit model
summary(logreg_single)$coeff

The `predict` function makes predictions using our fitted model. It has two arguments:
* the fitted logistic regression model
* `type` = the format of our predictions. By selecting "response", we will get back a probability between 0 (i.e. no predicted chance of attrition) and 1 (100% predicted chance of attrition)

In [ ]:
# use the model to predict attrition
single_probs = predict(logreg_single, hr, type = "response")

# show the probabilities of attrition for the first six employees in the dataset
head(single_probs)

We can view the range of predicted probabilities with a box plot. 

In [ ]:
# show the range of predictions
boxplot(single_probs)

Having a predicted probability is interesting, but often we want our predictions to be in the same form as our original `Attrition` variable: either "Yes" or "No" values. This allows us to directly measure the accuracy of our model by comparing our predictions with the true outcomes.  

There are a few ways to transform our probabilities into "Yes"/"No" values:

- We might decide that any probablity above a certain threshold will be coded as "Yes". For example, if the predicted probability is at least 0.24, we would predict attrition.
- If we believe that a certain proportion of staff are likely to leave the company, we might decide to code the top $n$ percent of predictions as "Yes". For example, if we know that last year we lost 5% of employees to attrition, we could predict attrition this year for the 5% of employees with the highest predicted probability.

<div class="alert alert-warning">
    <b>QUESTION</b>: What method do you think is best for interpreting the probabilities, and why? You may suggest a different method from the two listed above.
    </div>


*YOUR ANSWER HERE*

In this example, we'll try coding any employee with a probability of at least 0.22 as likely to leave.

In [ ]:
# create a categorical variable: "Yes" if probability of attrition is > 0.22, "No" otherwise
single_preds = ifelse(single_probs > 0.22, "Yes", "No")

# show the first six items in the probability and prediction vectors for our single variable model
head(data.frame(single_probs, single_preds))

One way to check our accuracy is to calculate how many predictions were correct out of the total number of predictions. This will take two steps:
1. for each prediction, check if it equals the actual value (`single_preds == hr[, "Attrition"]`)
2. add up the number of correct predictions and divide it by the total predictions (`mean`)

The resulting number is the proportion of correct predictions.

In [ ]:
# show the proportion of correct predictions
mean(single_preds == hr[, "Attrition"])

It can also be useful to see in what ways our model is correct or incorrect. We can visualize this with a **confusion matrix**. Each row shows the instances of the predicted attrition, and each column shows the instances of the actual attrition. The matrix values are the counts. So:
- the value in row one, column one is the *true negatives*: the number of employees who were predicted to stay at the company and actually did stay
- the value in row one, column two is the *false negatives*: number of employees who were predicted to stay and actually left 
- the value in row two, column one is the *false positives*: employees who were predicted to leave but didn't
- the value in row two, column two is the *true positives*: employees who were predicted to leave and did

<img src="images/confusion-matrix.jpg" />

In [ ]:
# create a confusion matrix for our predictions (rows) vs the actual values (columns)
table(single_preds, hr[, "Attrition"])


Another metric often used to evaluate how good the model fits the data is the R-squared (R^2), which summarizes the proportion of variance in the dependent variable associated with the independent variables. Larger R-squared values indicate that more of the variation is explained by the model, to a maximum of 1. R-squared is generally used on continuous data, however there is a modified version of R-squared (Nagelkerke) that we can use to evaluate our binary dependent variable in the logistic regression model.

In [ ]:
NagelkerkeR2(logreg_single)$R2

<div class="alert alert-warning">
    <b>QUESTION</b>: How would you rate the accuracy of this model? Would you recommend this model be used in the company, and why or why not? What is a "good" accuracy for a model like this, and why might it be important to decide that before conducting analysis? What are some pros and cons to this model (i.e. using Monthly Income to predict attrition), and what are possible assumptions and biases associated with it? What does the Nagelkerke R-squared value suggest?
    </div>

*YOUR ANSWER HERE*

### <a id="sectionex2"> Example 2: Predict By Monthly Income, Work-Life Balance, and Years at Company (multivariate logistic regression) </a>

Perhaps we could do better if we used more predictor variables. In this example, we'll attempt to predict attrition based on the employee's monthly income, work-life balance, and number of years with the company.

The nice thing about using a programming language like R is that the large majority of the code used for the univariate model will work for a multivariate model. The main thing that needs to change is the initial formula fed into the model.

As before, our first step is to write the formula and give it a name. Here, the name of the formula variable is `multivar_formula`, and the additional predictor variables are added using `+`. So, for a formula with $n$ predictor variables, the formula would look like this:

$$ \text{response variable} \sim \text{predictor variable}_1 + \text{predictor variable}_2 + \text{...} + \text{predictor variable}_n $$


In [ ]:
multi_formula = AttritionInt ~ MonthlyIncome + WorkLifeBalance + YearsAtCompany

The rest of the steps are almost identical to our single variable model:

1. **fit the model** to the data using the multivariate logistic regression formula
2. use the fitted model to **predict** attrition for each employee
3. **evaluate the model** by looking at summary data, the proportion of correct predictions, the confusion matrix, R-squared and plots of the results

In [ ]:
# fit the model
logreg_multi = glm(multi_formula, 
              data = hr, # the dataset
              family = binomial) # the type of model to use

# use the model to predict attrition
multi_probs = predict(logreg_multi, hr, type = "response")

# create a categorical variable: "Yes" if probability of attrition is > 0.22, "No" otherwise
multi_preds = ifelse(multi_probs > 0.22, "Yes", "No")

In [ ]:
# show the proportion of correct predictions
mean(multi_preds == hr[, "Attrition"])

In [ ]:
NagelkerkeR2(logreg_multi)$R2

<div class="alert alert-warning">
    <b>QUESTION</b>: Why wouldn't we want to just use all the variables as predictors?
    </div>

*YOUR ANSWER HERE*

---

## <a id="sectionmake"> 4. Make Your Own Model </a>

Now that you've seen some examples, try making and evaluating your own model. Most of the code has been provided for you (you might notice that it looks extremely similar to the univariate model). However, you will need to do the following:

1. Choose at least one variable as a predictor (and justify your choice)
2. Run the cells to train the model and generate probablities
3. Choose a method for interpreting attrition probabilities (and justify your choice)
4. Run the cells to generate model metrics and visualizations, and analyze your model

### Write the formula

Start by writing the formula for your logistic regression model.

Remember, the syntax for the formula is: 

$$ \text{response variable} \sim \text{predictor variable}_1 + \text{predictor variable}_2 + \text{...} + \text{predictor variable}_n $$

where the predictor variables are the names of the columns you want to use (spelling and capitalization count!)

You may want to revisit the <a id="sectionEDA">Exploratory Analysis</a> section to refresh your memory on the values and relationships of the possible predictor variables. You can also quickly see all possible variable names by running the following cell:

In [ ]:
# Run this cell if you want a list of all possible variable names
names(hr)

<div class="alert alert-warning">
    <b>QUESTION</b>: Complete the formula with your chosen predictor variables.
    </div>

In [ ]:
# write the formula for your logistic regression model by replacing the ...
my_formula = AttritionInt ~ ...

<div class="alert alert-warning">
    <b>QUESTION</b>: Which predictor variables did you choose, and why?
    </div>

*YOUR ANSWER HERE*

### Fit the model

Once your formula is complete, run the following cell to fit your model.

If you get an error, double-check that you've run the cell that contains your formula, and that all the predictor variable names are spelled correctly.

In [ ]:
# fit the model
logreg_mine = glm(my_formula, 
              data = hr, # the dataset
              family = binomial) # the type of model to use

Use the `summary` function to view some information about the fit model.

### Make predictions

In [ ]:
# use the model to predict attrition
my_probs = predict(logreg_mine, hr, type = "response")

# show the probabilities of attrition for the first six employees in the dataset
head(my_probs)

View the range of predicted probabilities with a box plot. 

In [ ]:
# show the range of predictions
boxplot(my_probs)

<div class="alert alert-warning">
    <b>QUESTION</b>: Decide how to turn your probabilities into "Yes"/"No" predictions.
    <ul>
        <li> If you want to use the threshold method, set the <code>method</code> variable to "threshold" and set the <code>cutoff</code> variable to the minimum probability you would like to be coded as "Yes" (e.g. <code>cutoff = 0.4</code> would code all probabilities of 0.4 or higher as "Yes"</li>
        <li> If you want to use the percentile method, set the <code>method</code> variable to "percentile", and set the <code>percentile</code> variable to the percentile above which you want to code as "Yes" (e.g if you want the bottom 95% of predictions to be "No" and the top 5% of predictions to be "Yes", <code>percentile=0.95</code></li>
    </ul>
    </div>

In [ ]:
# replace the ... with either "threshold" or "percentile"
method = ...

# if you're using the threshold method, replace the ... with the 
# cutoff probability (a number between 0 and 1)
cutoff = ...

# if you're using the percentile method, replace the ... with
# the dividing percentile (a number between 0 and 1)
percentile = ...

# create a categorical variable: "Yes" if probability of attrition is > 0.23, "Yes" otherwise
if (method == "threshold") {
    my_preds = ifelse(my_probs > cutoff, "Yes", "No")
} else if (method == "percentile") {
    pct_cutoff = quantile(my_probs, percentile)
    my_preds = ifelse(my_probs > pct_cutoff, "Yes", "No")
} else {
    stop("method variable must be either \"threshold\" or \"percentile\"")
}

# show the first six items in the probability and prediction vectors for our single variable model
head(data.frame(my_probs, my_preds))

In [ ]:
# show some summary information about the fit model
summary(logreg_mine)

Calculate the proportion of correct predictions.

In [ ]:
# show the proportion of correct predictions
mean(my_preds == hr[, "Attrition"])

Create the confusion matrix. Remember, the format is:

<img src="images/confusion-matrix.jpg" />

In [ ]:
# create a confusion matrix for our predictions (rows) vs the actual values (columns)
table(my_preds, hr[, "Attrition"])


In [ ]:
NagelkerkeR2(logreg_mine)$R2

<div class="alert alert-warning">
    <b>QUESTION</b>: How would you rate the accuracy of this model? Would you recommend this model be used in the company's HR department, and why or why not? What is a "good" accuracy for a model like this, and why might it be important to decide that before conducting analysis? Out of the 3 models we developed, which one do you prefer and why? 
    </div>

*YOUR ANSWER HERE*

## <a id= "sectionsubmit"> Submitting your assignment </a>

After you have finished the notebook, you will download it as a pdf file and upload it to the bCourses site. To download notebooks as pdfs:

1. Make sure you've saved your work by clicking the floppy disc icon at the top left or going to the "File" menu and clicking "Save and Checkpoint".
2. Once you've saved your notebook, go to the "File" menu at the top left of the notebook screen.
2. Click "Download as", then click "PDF vis LaTeX (.pdf)".
3. Depending on how your internet browser is set up to handle PDF files, your download may start automatically or you may be asked where you want to save the file.

Once the file is downloaded to your computer, you may upload it to bCourses for credit.

<div class="alert alert-info"><b>NOTE</b>: Make sure to check that your answers appear in the pdf file (you can quickly find the places where you had to fill in text by using control+F (Windows) or command + F (Mac) to search for "Question"). If your answers don't appear, you may not have saved your notebook before downloading. If you run into other issues, don't hesitate to ask your classmates, Daniela, or the Peer Consultants for help!</div>

----------

Notebook created by <a href="mailto: heather.whiteman@berkeley.edu">Heather Whiteman</a>, <a href="mailto: ktakimoto@berkeley.edu">Keeley Takimoto</a> and Ignacio Solis</a>.

Some text adapted with permission from [materials made for Haas Executive Education's Data Science Online course by Keeley Takimoto](https://github.com/ktakimoto/haas-ds-online).